# App Store Data Challenge

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
!ls ../challenge_data/

Data challenge instructions - AR.pdf device_ref.csv
account_dat.csv                      in-app_dat.csv
app_dat.csv                          transaction_dat.csv
category_ref.csv


In [3]:
df_account = pd.read_csv('../challenge_data/account_dat.csv')
df_app = pd.read_csv('../challenge_data/app_dat.csv')
df_category = pd.read_csv('../challenge_data/category_ref.csv')
df_device = pd.read_csv('../challenge_data/device_ref.csv')
df_in_app = pd.read_csv('../challenge_data/in-app_dat.csv')
df_transaction = pd.read_csv('../challenge_data/transaction_dat.csv')

In [4]:
df_account.head()

,acct_id,create_dt,payment_type
0,eb23e449d274bddb,4/3/12,PMOF
1,3d60dcb8494a0d70,8/31/09,PMOF
2,9b6d6fbf76504d85,11/6/08,PMOF
3,343255370828b380,1/3/12,PMOF
4,664e27f05189c387,10/31/10,PMOF


In [5]:
df_app.head()

,app_name,content_id,category_id,device_id
0,half-slouched having,22dfac9721b899cb,Y-8,3
1,lift whale-ships,7c645810cd7f166e,M-7,3
2,mightily community,c495bcb878faffaa,Y-8,2
3,session: too--shall,4e683dcd461cdd22,Y-8,1
4,That Zoology,6473dd29a2badbf1,Y-8,3


In [6]:
df_in_app.head()

,parent_app_content_id,content_id,type
0,d2bde35599e0dae9,227227a77c59809b,consumable
1,d2bde35599e0dae9,41ac2362a2600185,consumable
2,d2bde35599e0dae9,64a8e16f030bb611,consumable
3,d2bde35599e0dae9,2c8ba424183afa7c,consumable
4,d2bde35599e0dae9,b4c42fa5cc420f7b,consumable


In [7]:
df_category.head()

,category_name,category_id
0,Games,Z-1
1,Social Networking,Q-5
2,Utilities,M-7
3,Entertainment,Q-1
4,Photos & Videos,Y-8


### Denormalize data 3 dataframes
1. df_app_denorm: dataframe with info on category, content, and device
2. df_trn_denorm: dataframe with transactions, account, and device
3. df_wide: dataframe with all columns

In [8]:
# cast date columns as datetime objects
df_account['create_dt'] = pd.to_datetime(df_account['create_dt'])
df_transaction['create_dt'] = pd.to_datetime(df_transaction['create_dt'])

df_app_denorm = df_in_app.join(df_app.set_index('content_id'), on='parent_app_content_id')\
                .join(df_category.set_index('category_id'), on='category_id')\
                .join(df_device.set_index('device_id'), on='device_id')

df_trn_denorm = df_transaction.join(df_device.set_index('device_id'), on='device_id')\
                    .join(df_account.set_index('acct_id'), on='acct_id', lsuffix='_transaction', rsuffix='_acct')

df_wide = df_trn_denorm.join(df_app_denorm.drop(columns=['device_id','device_name'])\
                             .set_index('content_id'), on='content_id')

In [9]:
df_app_denorm.head()

,parent_app_content_id,content_id,type,app_name,category_id,device_id,category_name,device_name
0,d2bde35599e0dae9,227227a77c59809b,consumable,lactantem prevailing,Z-1,3,Games,Both
1,d2bde35599e0dae9,41ac2362a2600185,consumable,lactantem prevailing,Z-1,3,Games,Both
2,d2bde35599e0dae9,64a8e16f030bb611,consumable,lactantem prevailing,Z-1,3,Games,Both
3,d2bde35599e0dae9,2c8ba424183afa7c,consumable,lactantem prevailing,Z-1,3,Games,Both
4,d2bde35599e0dae9,b4c42fa5cc420f7b,consumable,lactantem prevailing,Z-1,3,Games,Both


In [10]:
df_trn_denorm.head()

,create_dt_transaction,content_id,acct_id,price,device_id,device_name,create_dt_acct,payment_type
0,2016-08-28,b6cf4f33e7455df4,220574a7a6a71afa,11.99,1,iPhone,2009-08-29,PMOF
1,2016-09-16,71ac3e9c38ddbbba,8551f69a5015ea6a,1.99,2,iPad,2011-05-28,PMOF
2,2016-08-15,776b851b916c33d1,7dedf309494a1966,0.99,2,iPad,2009-11-15,PMOF
3,2016-06-17,7a47ca132a8dc695,7174a81c796dcf7e,1.99,2,iPad,2009-03-01,PMOF
4,2016-06-13,30581219bc952dbb,35aaf59d64a677e7,11.99,2,iPad,2010-10-01,PMOF


In [11]:
df_wide.head()

,create_dt_transaction,content_id,acct_id,price,device_id,device_name,create_dt_acct,payment_type,parent_app_content_id,type,app_name,category_id,category_name
0,2016-08-28,b6cf4f33e7455df4,220574a7a6a71afa,11.99,1,iPhone,2009-08-29,PMOF,61cea77b9bc843d1,subscription,events Bildad's,M-7,Utilities
1,2016-09-16,71ac3e9c38ddbbba,8551f69a5015ea6a,1.99,2,iPad,2011-05-28,PMOF,1f0ebd67df5846cd,consumable,swell questionably,M-7,Utilities
2,2016-08-15,776b851b916c33d1,7dedf309494a1966,0.99,2,iPad,2009-11-15,PMOF,1383e4f2f49f0f44,consumable,supperless--my Shipmates,Z-1,Games
3,2016-06-17,7a47ca132a8dc695,7174a81c796dcf7e,1.99,2,iPad,2009-03-01,PMOF,8590a25d4e5c249c,consumable,unwritten hungry,Z-1,Games
4,2016-06-13,30581219bc952dbb,35aaf59d64a677e7,11.99,2,iPad,2010-10-01,PMOF,478e10f9248b7842,consumable,Turning thanked,Q-1,Entertainment


In [12]:
df_wide.create_dt_acct

0         2009-08-29
1         2011-05-28
2         2009-11-15
3         2009-03-01
4         2010-10-01
             ...    
3607504   2011-06-18
3607505   2010-01-25
3607506   2012-02-08
3607507   2007-08-29
3607508   2009-01-23
Name: create_dt_acct, Length: 3607509, dtype: datetime64[ns]